In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [5]:
design.delete_all_components()

In [6]:
design.get_chip_z('Q_chip')

'5um'

In [3]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='8um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.35
    resonator_length_2.append(a)
print(resonator_length_2)


[8.103254683870285, 7.874788339841421, 7.658346540209131]


# Flipchip

In [6]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-150}um",
                                              pos_y=f"{q0y+140}um",
                                              pad_width="150um",
                                              pad_height="30um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [7]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [8]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [11]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [12]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [13]:
hfss.start()

INFO 10:20AM [connect_project]: Connecting to Ansys Desktop API...
INFO 10:20AM [load_ansys_project]: 	Opened Ansys App
INFO 10:20AM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 10:20AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project1
INFO 10:20AM [connect_design]: No active design found (or error getting active design).
INFO 10:20AM [connect]: 	 Connected to project "Project1". No design detected


True

In [14]:
hfss.activate_ansys_design("flip8um_V1", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

10:20AM 25s WARNING [activate_ansys_design]: The design_name=flip8um_V1 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 10:20AM [connect_design]: 	Opened active design
	Design:    flip8um_V1 [Solution type: Eigenmode]
WARNING 10:20AM [connect_setup]: 	No design setup detected.
WARNING 10:20AM [connect_setup]: 	Creating eigenmode default setup.
INFO 10:20AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [15]:
hfss.close()

True

# LOM Analysis

In [12]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [13]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.31',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.31',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [15]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 12:58PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpnhcjyneo.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5310000000, Maxwell, 1, False
INFO 12:58PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp92v7k07v.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5310000000, Maxwell, 1, False
INFO 12:58PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp4qbfvs1y.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5310000000, Maxwell, 2, False
INFO 12:58PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpeoobjx8t.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5310000000, Maxwell, 3, False
INFO 12:58PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpu3j5xo7y.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5310000000, Maxwell, 4, False
INFO 12:58

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,356.55640,-112.14530,-110.84780,-66.18471
pad_bot_Q0,-112.14530,141.54900,-28.43855,-0.20587
pad_top_Q0,-110.84780,-28.43855,141.29070,-1.83933
quad_coupler_c0,-66.18471,-0.20587,-1.83933,68.40218


In [16]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 7.27
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 4.919218 [GHz]
EC 223.416267 [MHz]
EJ 14.854145 [GHz]
alpha -250.920684 [MHz]
dispersion 0.601238 [KHz]
Lq 10.995564 [nH]
Cq 86.700169 [fF]
T1 16999.216947 [us]

**Coupling Properties**

tCqbus1 -0.815200 [fF]
gbus1_in_MHz -10.451785 [MHz]
χ_bus1 -0.009340 [MHz]
1/T1bus1 9.362487 [Hz]
T1bus1 16999.216947 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.487632,281.508499,14.854145,-321.833949,6.896523,[-52.00236708885216],[-0.47508377119728423],0.475084,52.002367
2,5.0916,240.241825,14.854145,-271.237906,1.335841,[-17.545208403823924],[-0.032411094922803516],0.032411,17.545208
3,4.976517,228.933482,14.854145,-257.562904,0.788847,[-9.848660577026264],[-0.008880227321555481],0.008880,9.848661
4,4.958164,227.158214,14.854145,-255.423519,0.72363,[-8.287865333039418],[-0.00615188102875095],0.006152,8.287865
5,4.919069,223.401984,14.854145,-250.903514,0.600807,[-10.291618317297246],[-0.009054548889686908],0.009055,10.291618
6,4.919218,223.416267,14.854145,-250.920684,0.601238,[-10.451785297167907],[-0.009340226212375421],0.009340,10.451785


In [ ]:
c1.sim.close()